# Synthetic Sinusoidal Traces Plotting/Analysis

This notebook examine in depth the `sythetic-sinusoidal` input rate generation method in the `dfaas_input_rate` module.

Some constraints (or assumptions):

1. Each episode has a fixed length of 288 steps (each step represents 5 minutes in a 24-hour period).
2. For each step and agent, the input rate takes positive values in a fixed range (1, 150).

In [ ]:
# Common imports.
from pathlib import Path
from copy import deepcopy

%matplotlib widget
import base

import dfaas_input_rate

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
min_reqs, max_reqs = 0, 150  # Range of the generated requests.
max_steps = 288  # Steps of a single episode (see the DFaaS env).

In [ ]:
def scale_down(traces, max_per_agent=63, min_rate_per_agent=1, max_rate_per_agent=150):
    agents = list(traces.keys())
    max_steps = len(next(iter(traces.values())))  # All traces have the same length, just get the first.
    total_capacity_step = max_per_agent * len(agents)
    scaled_traces = deepcopy(traces)

    for step in range(max_steps):
        input_rate_step = np.array([traces[agent][step] for agent in agents])
        if input_rate_step.sum() <= total_capacity_step:
            continue  # Capacity not exceeded, nothing to do.

        # Traces for this step must be scaled down proportionally.
        scaled = np.round(input_rate_step * total_capacity_step / input_rate_step.sum())

        # Clip values to ensure they're in the correct range.
        scaled = np.clip(scaled, min_rate_per_agent, max_rate_per_agent).astype(int)

        # Adjust single input rates to match total_capacity_step. Rounding and
        # clipping can cause the sum to be slightly off.
        diff = total_capacity_step - scaled.sum()
        while diff != 0:
            if diff > 0:
                # Add to agents below max_rate_per_agent.
                for i in range(len(scaled)):
                    if scaled[i] < max_rate_per_agent:
                        scaled[i] += 1
                        diff -= 1
                        if diff == 0:
                            break
            else:
                # Subtract from agents above min_rate_per_agent.
                for i in range(len(scaled)):
                    if scaled[i] > min_rate_per_agent:
                        scaled[i] -= 1
                        diff += 1
                        if diff == 0:
                            break

        # Update the scaled_traces with the new rates. Each agent is mapped to
        # an integer (the orders is the same across all iterations).
        for agent, idx in zip(agents, range(len(agents)), strict=True):
            scaled_traces[agent][step] = scaled[idx]

    return scaled_traces

In [ ]:
def synthetic_sinusoidal(max_steps, agents, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    if max_steps <= 0:
        raise ValueError(f"Expected > 0, found {max_per_agent = }!")

    # All agents have the same basline_rate and amplitude_rate.
    baseline_rate = np.repeat(70, len(agents))
    amplitude_rate = np.repeat(65, len(agents))
    noise_ratio = rng.uniform(0.05, 0.1, len(agents))  # But different noise ratio.
    steps = np.arange(max_steps)
    rate_min, rate_max = 1, 150

    # Avoid overlapping phases (that may cause a global overload) by evenly
    # spacing the phase for each agent.
    phi = np.linspace(0, 2 * np.pi, len(agents), endpoint=False)

    # Generate the input rate traces for each agent.
    input_rates = []
    for idx in range(len(agents)):
        base_rate = amplitude_rate[idx] * np.sin(2 * np.pi * steps / max_steps + phi[idx]) + baseline_rate[idx]

        noisy_rate = base_rate + noise_ratio[idx] * rng.normal(0, amplitude_rate[idx], size=max_steps)

        clipped_rate = np.clip(np.round(noisy_rate), rate_min, rate_max)

        input_rates.append(clipped_rate)

    # Randomly assign an input rate trace for each agent.
    traces = {}
    for agent, input_rate in zip(agents, rng.permutation(input_rates)):
        traces[agent] = input_rate

    return traces

In [ ]:
networks = [
    ["node_0", "node_1"],
    ["node_0", "node_1", "node_2"],
    ["node_0", "node_1", "node_2", "node_3"],
    ["node_0", "node_1", "node_2", "node_3", "node_4"],
]

for x in range(50):
    max_steps = 288
    agents = networks[np.random.randint(0, len(networks))]
    traces = synthetic_sinusoidal(max_steps, agents)
    scaled = scale_down(traces)

In [ ]:
def plot_traces(traces, scaled, suffix=None):
    for agent, trace in traces.items():
        plt_id = f"trace_{agent}" if not suffix else f"trace_{agent}_{suffix}"
        plt.close(fig=plt_id)
        fig = plt.figure(num=plt_id, layout="constrained")
        fig.canvas.header_visible = False
        ax = fig.subplots()

        ax.plot(trace, label="Original")
        ax.plot(scaled[agent], label="Scaled")

        title = f"Function rate ({agent = })"
        if suffix:
            title += f" ({suffix})"

        ax.set_title(title)
        ax.set_ylabel("Invocations")
        ax.set_ylim(bottom=0, top=150 + 5)
        ax.set_xlabel("Step")

        ax.legend()
        ax.grid(axis="both")
        ax.set_axisbelow(True)


max_steps = 288
agents = ["node_0", "node_1"]
# agents = ["node_0", "node_1", "node_2", "node_3"]
traces = synthetic_sinusoidal(max_steps, agents)
scaled = scale_down(traces)
plot_traces(traces, scaled)